In [1]:
import os
import sklearn.linear_model
import sklearn.tree
import sklearn.metrics
import time
import warnings

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from MLPClassifierWithSolverLBFGS import MLPClassifierLBFGS
from viz_tools_for_binary_classifier import plot_pretty_probabilities_for_clf

from sklearn import tree
from sklearn.model_selection import KFold

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
import spacy
from spacy.lang.en import English
stoplist = spacy.lang.en.stop_words.STOP_WORDS

def ispunct(j):
    if j == '.' or j == '!' or j == ',' or j == '[]' or j == ']' or j == '(' or j == ')' \
         or j == '-' or j == '$' or j == '%' or j == '``':
        return True
    else:
        return False

In [3]:
x_train_df = pd.read_csv('data_reviews/x_train.csv')
y_train_df = pd.read_csv('data_reviews/y_train.csv')
#x_test_df  = pd.read_csv('data_reviews/x_test.csv', encoding='latin-1')

y_list = y_train_df['is_positive'].values.tolist()
the_list = x_train_df['review'].values.tolist()

In [4]:
i=0
tr_text_list = the_list
edited_list = tr_text_list
for i in range(0,len(tr_text_list)):
    #print(tr_text_list[i])
    edited_list[i] = tr_text_list[i].lower()
    edited_list[i] = ''.join([j for j in tr_text_list[i] if (not j.isdigit() and not ispunct(j))])
    edited_list[i] = tr_text_list[i].strip()
    tokens = word_tokenize(tr_text_list[i])
    edited_list[i] = [j for j in tokens if not j in stoplist]
    
#for i in range(0,100):
    #print(edited_list[i])
    
    


In [5]:
train_edited = edited_list[:2160]

test_edited = edited_list[2160:]
y_test = y_list[2160:]



In [6]:
pos_words = ""
neg_words = ""
cat = pos_words + train_edited[0][0]
#print(y_list[0])
#print(edited_list[0][0])
for i in range(0,len(train_edited)):
    if y_list[i] == 0:
        for j in range(0,len(train_edited[i])):
            neg_words = neg_words + train_edited[i][j] + " "
    else:
        for j in range(0,len(train_edited[i])):
            pos_words = pos_words + train_edited[i][j] + " "
    
#print(pos_words)

In [7]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [20]:
#corp_pos = [line.replace("", "") for line in pos_words]
corp_pos = pos_words.split()
corp_neg = neg_words.split()
#print(corp_pos)

num = 1500

#print("Top positive words: ")
top_pos = get_top_n_words(corp_pos, num)
#for i in range(0,num):
#    print(top_pos[i])
    
#print("\n")
#print("Top negative words: ")
top_neg = get_top_n_words(corp_neg, num)
#for i in range(0,num):
#    print(top_neg[i])
    

In [21]:
pos = []
neg = []

num_top_pos = int( len(top_pos)/1 )
num_top_neg = int( len(top_neg)/1 )
freq_factor = 10

#print(num_top_pos)
#print(top_pos[i][1])
#print(top_pos[i][1] * 2)

for i in range(0,num_top_pos):
    repeated = False
    for j in range(0,num_top_neg):
        if top_pos[i][0] == top_neg[j][0]:
            if top_pos[i][1] < freq_factor*top_neg[j][1]:
                #print("word: ", top_pos[i][0])
                #print("pos times: ", top_pos[i][1], ", neg times: ", top_neg[j][1])
                #print('repeated word: ', top_pos[i][0], " ", top_neg[j][0])
                repeated = True
            
    if not repeated:
        pos.append(top_pos[i][0])
            
for i in range(0,num_top_neg):
    repeated = False
    for j in range(0,num_top_pos):
        if top_neg[i][0] == top_pos[j][0]:
            if top_neg[i][1] < freq_factor*top_pos[j][1]:
                #print("word: ", top_pos[i][0])
                #print("pos times: ", top_pos[i][1], ", neg times: ", top_neg[j][1])
                #print('repeated word: ', top_pos[i][0], " ", top_neg[j][0])
                repeated = True
            
    if not repeated:
        neg.append(top_neg[i][0])
        
#print("Pos words: ", pos, "\n")

#print("Neg words: ", neg)

In [22]:
guesses = np.zeros(len(y_test))

for i in range(0,len(test_edited)):
    for word in test_edited[i]:
        if word in pos:
            guesses[i] = guesses[i] + 1
        elif word in neg:
            guesses[i] = guesses[i] - 1
            
print(guesses)

[ 2.  1.  1. -1.  0.  1.  0.  0.  0. -1.  0.  0.  1.  0.  1.  2.  0.  0.
  3. -3.  0.  0.  1.  0.  0.  1.  0. -2.  1.  0.  0.  0.  1. -1.  1. -1.
  2.  0.  3.  1. -1.  0.  0.  0.  0.  1. -2.  3.  1.  1.  2.  1.  2.  2.
  1. -1.  0.  0. -1.  0.  1.  1.  1.  0.  0.  0.  1. -2. -1.  2.  0.  1.
  1.  0. -2.  0.  0.  1.  0.  0.  0.  0.  2.  1.  0. -1.  1.  1.  2. -1.
  0.  0.  0.  0. -1.  0.  0.  1.  0.  1.  1.  1.  0. -1.  0.  0. -1.  0.
 -1.  1. -1. -1.  0.  0. -1.  1. -1.  1.  0. -1. -1.  1.  0. -2.  1.  1.
  0.  0.  1. -1.  0.  0.  0.  0.  1.  0.  1.  1.  0.  0.  1. -1.  1.  0.
  0.  1. -2.  0. -1.  1.  1.  1.  0.  1. -1.  1.  2.  0.  1.  0.  2.  1.
  0.  2.  0.  1.  1.  1.  2. -1. -1. -1. -1.  0.  0.  1.  1. -2.  2.  1.
 -1.  0. -1. -1.  0.  0.  1.  1.  2.  0.  2.  1.  1.  0. -2.  0.  1.  0.
 -2.  3.  1.  0.  0.  1.  0.  1.  1.  0.  1.  0.  3.  0.  0.  0. -2.  2.
 -1.  0. -1.  1. -2.  0.  1.  1.  0.  1.  0. -1.  0.  1.  0.  1. -1.  2.
  1.  0. -1.  1.  1.  0.]


In [23]:
correct = 0
incorrect = 0
unsure = 0

for i in range(0,len(test_edited)):
    if guesses[i] > 0:
        if y_test[i] == 1: 
            correct = correct + 1
        else: 
            incorrect = incorrect + 1
    elif guesses[i] < 0:
        if y_test[i] == 0: 
            correct = correct + 1
        else: 
            incorrect = incorrect + 1
            
    else:
        unsure = unsure + 1
            
print("correct: ", correct)     
print("incorrect: ", incorrect)  
print("pct correct: ", (correct / (incorrect + correct) * 100))
print("unsure: ", unsure)  
            

correct:  98
incorrect:  48
pct correct:  67.12328767123287
unsure:  94
